# aula 19:

## Setando a raiz do projeto:

In [12]:
import os 
os.getcwd()
os.chdir("/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_19")
os.getcwd()

'/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_19'

## Desafio

Criar nosso primeiro CRUD

1. **`POST /items/`: Cria um novo item**
    
    Esta rota permite criar um novo item no banco de dados. O cliente envia os dados do novo item no corpo da solicitação HTTP e o servidor adiciona esse item ao banco de dados. Aqui está como funciona:
    
    * **Verbo HTTP**: POST
    * **Endpoint**: `/items/`
    * **Ação**: Cria um novo item no banco de dados.
    * **Requisitos**: O corpo da solicitação deve conter os dados do novo item.
    * **Resposta**: Retorna o novo item criado.
2. **`GET /items/`: Retorna uma lista paginada de itens**
    
    Esta rota permite recuperar uma lista paginada de itens do banco de dados. O cliente pode especificar opcionalmente os parâmetros `skip` (quantos itens pular) e `limit` (quantos itens retornar) para paginação. Aqui está como funciona:
    
    * **Verbo HTTP**: GET
    * **Endpoint**: `/items/`
    * **Ação**: Retorna uma lista paginada de itens do banco de dados.
    * **Parâmetros de consulta**: `skip` (opcional, padrão = 0) e `limit` (opcional, padrão = 10).
    * **Resposta**: Retorna uma lista de itens conforme especificado pelos parâmetros de consulta.
3. **`GET /items/{item_id}`: Retorna um item específico com base no ID**
    
    Esta rota permite recuperar um item específico do banco de dados com base no ID fornecido. Aqui está como funciona:
    
    * **Verbo HTTP**: GET
    * **Endpoint**: `/items/{item_id}`
    * **Ação**: Retorna um item específico com base no ID fornecido.
    * **Parâmetros de caminho**: `item_id` (ID do item a ser recuperado).
    * **Resposta**: Retorna o item correspondente ao ID fornecido.
4. **`PUT /items/{item_id}`: Atualiza um item existente com base no ID**
    
    Esta rota permite atualizar os dados de um item existente no banco de dados com base no ID fornecido. O cliente envia os novos dados do item no corpo da solicitação HTTP. Aqui está como funciona:
    
    * **Verbo HTTP**: PUT
    * **Endpoint**: `/items/{item_id}`
    * **Ação**: Atualiza um item existente com base no ID fornecido.
    * **Parâmetros de caminho**: `item_id` (ID do item a ser atualizado).
    * **Requisitos**: O corpo da solicitação deve conter os novos dados do item.
    * **Resposta**: Retorna o item atualizado.
5. **`DELETE /items/{item_id}`: Exclui um item existente com base no ID**
    
    Esta rota permite excluir um item existente no banco de dados com base no ID fornecido. Aqui está como funciona:
    
    * **Verbo HTTP**: DELETE
    * **Endpoint**: `/items/{item_id}`
    * **Ação**: Exclui um item existente com base no ID fornecido.
    * **Parâmetros de caminho**: `item_id` (ID do item a ser excluído).
    * **Resposta**: Retorna o item excluído.

Essas operações fornecem uma API completa para gerenciar itens no banco de dados, permitindo criar, recuperar, atualizar e excluir itens de forma eficiente e segura. Certifique-se de que as operações estejam de acordo com os requisitos do seu projeto e que você implemente a lógica necessária para garantir a consistência e a segurança dos dados.

### `models.py`:

In [8]:
%%writefile src/models.py
from sqlalchemy import Column, Float, Integer, String
from sqlalchemy.orm import declarative_base

# Criação de uma classe base para a definição de modelos
Base = declarative_base()

class Item(Base):
    """
    Modelo para a tabela 'items'.

    Atributos:
        id (int): Identificador único do item (chave primária).
        name (str): Nome do item.
        price (float): Preço do item.
        is_offer (str): Indica se o item está em oferta (opcional).
    """
    __tablename__ = "items"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    price = Column(Float)
    is_offer = Column(String, nullable=True)


Overwriting src/models.py


### `Schema.py`:

In [9]:
%%writefile src/schema.py
from typing import Union
from pydantic import BaseModel

class ItemBase(BaseModel):
    """
    Classe base para definição de atributos comuns de um item.

    Atributos:
        name (str): Nome do item.
        price (float): Preço do item.
        is_offer (Union[bool, None], opcional): Indica se o item está em oferta.
    """
    name: str
    price: float
    is_offer: Union[bool, None] = None

class ItemCreate(ItemBase):
    """
    Classe para criar um novo item.

    Herda os atributos da classe ItemBase.
    """
    pass

class Item(ItemBase):
    """
    Classe que representa um item.

    Herda os atributos da classe ItemBase e adiciona um identificador único (id).
    
    Atributos:
        id (int): Identificador único do item.
    """
    id: int


Overwriting src/schema.py


### `database.py`:

In [7]:
%%writefile src/data.py
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# URL do banco de dados SQLite
SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"

# Criação de uma engine para o banco de dados
engine = create_engine(SQLALCHEMY_DATABASE_URL)

# Criação de uma classe base para a definição de modelos
Base = declarative_base()

# Criação de uma fábrica de sessões para interação com o banco de dados
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def get_db():
    """
    Função utilitária para obter uma sessão de banco de dados.

    Retorna:
        Uma sessão de banco de dados.
    """
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


Overwriting src/data.py


### `main.py`:

In [10]:
%%writefile src/main.py
from typing import List
from fastapi import Depends, FastAPI, HTTPException
from sqlalchemy.orm import Session

import data
import models
from schema import Item, ItemCreate

app = FastAPI()

# Criar as tabelas no banco de dados
models.Base.metadata.create_all(bind=data.engine)

@app.post("/items/", response_model=Item)
def create_item(item: ItemCreate, db: Session = Depends(data.get_db)):
    """
    Cria um novo item no banco de dados.

    Args:
        item (ItemCreate): Os dados do item a ser criado.
        db (Session, opcional): Uma sessão do banco de dados.

    Returns:
        Item: O item criado.
    """
    db_item = models.Item(**item.dict())
    db.add(db_item)
    db.commit()
    db.refresh(db_item)
    return db_item

@app.get("/items/", response_model=List[Item])
def read_items(skip: int = 0, limit: int = 10, db: Session = Depends(data.get_db)):
    """
    Retorna uma lista de itens do banco de dados.

    Args:
        skip (int, opcional): O número de itens a pular.
        limit (int, opcional): O número máximo de itens a retornar.
        db (Session, opcional): Uma sessão do banco de dados.

    Returns:
        List[Item]: Uma lista de itens.
    """
    items = db.query(models.Item).offset(skip).limit(limit).all()
    return items

@app.get("/items/{item_id}", response_model=Item)
def read_item(item_id: int, db: Session = Depends(data.get_db)):
    """
    Retorna um item específico do banco de dados.

    Args:
        item_id (int): O ID do item a ser retornado.
        db (Session, opcional): Uma sessão do banco de dados.

    Returns:
        Item: O item especificado pelo ID.
    """
    db_item = db.query(models.Item).filter(models.Item.id == item_id).first()
    if db_item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    return db_item

@app.put("/items/{item_id}", response_model=Item)
def update_item(item_id: int, item: ItemCreate, db: Session = Depends(data.get_db)):
    """
    Atualiza um item no banco de dados.

    Args:
        item_id (int): O ID do item a ser atualizado.
        item (ItemCreate): Os novos dados do item.
        db (Session, opcional): Uma sessão do banco de dados.

    Returns:
        Item: O item atualizado.
    """
    db_item = db.query(models.Item).filter(models.Item.id == item_id).first()
    if db_item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    for key, value in item.dict().items():
        setattr(db_item, key, value)
    db.commit()
    db.refresh(db_item)
    return db_item

@app.delete("/items/{item_id}", response_model=Item)
def delete_item(item_id: int, db: Session = Depends(data.get_db)):
    """
    Deleta um item do banco de dados.

    Args:
        item_id (int): O ID do item a ser deletado.
        db (Session, opcional): Uma sessão do banco de dados.

    Returns:
        Item: O item deletado.
    """
    db_item = db.query(models.Item).filter(models.Item.id == item_id).first()
    if db_item is None:
        raise HTTPException(status_code=404, detail="Item not found")
    db.delete(db_item)
    db.commit()
    return db_item


Overwriting src/main.py


In [ ]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/home/jcnok/bootcamps/bootcamp-jornada-de-dados_2024/aula_19']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [54664] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "main".


In [2]:
!python src/main.py